In [1]:
# %pip install sympy
# %pip install numpy
# %pip install matplotlib
# %pip install einops

In [2]:
from drawing import draw_robot

from sympy import *
import numpy as np

In [3]:
# Define state variables
theta, x, y, l = symbols("theta x y l")
q = Matrix([theta, x, y, l])
theta_dot, x_dot, y_dot, l_dot = symbols("theta_dot x_dot y_dot l_dot")
qd = Matrix([theta_dot, x_dot, y_dot, l_dot])
theta_ddot, x_ddot, y_ddot, l_ddot = symbols("theta_ddot x_ddot y_ddot l_ddot")
qdd = Matrix([theta_ddot, x_ddot, y_ddot, l_ddot])

# Define control variables
tau, f = symbols("tau f")
u = Matrix([tau, f])

# Define system parameters
m_b, I, m_l, g = symbols("m_b I m_l, g")
p = Matrix([m_b, I, m_l, g])

# Defin POIs
r_1 = Matrix([x, y])  # COM of body
r_2 = Matrix([x + (l + 0.5) * cos(theta), y + (l + 0.5) * sin(theta)])  # COM of leg
r_3 = Matrix([x + (l + 1.0) * cos(theta), y + (l + 1.0) * sin(theta)])  # foot

g_hat = Matrix([0, -1])  # gravity direction vector


# Define utility functions
def ddt(expr):
    # Take the gradient of expr with respect to q
    # Then multiply by qd
    return expr.jacobian(q) * qd + expr.jacobian(qd) * qdd


def u2Q(u):
    # Convert control inputs to generalized forces
    return q.jacobian(Matrix([theta, l])).T * u

# Define the Lagrangian

In [4]:
T = (
    1 / 2 * ddt(r_1).T * m_b * ddt(r_1)
    + 1 / 2 * ddt(r_2).T * m_l * ddt(r_2)
    + 1 / 2 * I * ddt(Matrix([theta]))**2
)
V = m_b * g * g_hat.T * r_1 + m_l * g * g_hat.T * r_2

L = T - V

In [5]:
u_exp = ddt(L.jacobian(qd)) - L.jacobian(q).T

A = u_exp.jacobian(qdd) # Mass matrix
b = u_exp - A * qdd # Coriolis and gravity terms

In [6]:
A_func = lambdify((q, qd, p), A, "numpy")
b_func = lambdify((q, qd, p), b, "numpy")

In [7]:
q_vals = [-0.25, 0, 2, 0]
qd_vals = [1.0, 0, 0, 2]
p_vals = [1.0, 0.05, 0.1, 9.81]

A_calcd = A_func(q_vals, qd_vals, p_vals)
b_calcd = b_func(q_vals, qd_vals, p_vals)

print("A =\n", A_calcd)
print("b =\n", b_calcd)

A =
 [[ 0.075       0.0123702   0.04844562  0.        ]
 [ 0.0123702   1.1         0.          0.09689124]
 [ 0.04844562  0.          1.1        -0.0247404 ]
 [ 0.          0.09689124 -0.0247404   0.1       ]]
b =
 [[ -0.27525154]
 [  0.05051596]
 [-10.39106483]
 [  0.19270328]]


In [8]:
draw_robot(np.array(q_vals))

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]